# Information about this takehome

Hello! Thanks for taking the time to look at this takehome. Before you begin, let's discuss a little context. 


Since you have been selected for this takehome, you have passed an initial resume screen. We believe that you have the potential to be the right person for this role.

Much of your work as a data scientist at The Recycling Partnership will look like:

1. Analysis and acquisition of data.
1. Implement innovative data flows, including machine learning pipelines.
1. Work with others to scope out data needs and solutions.
1. Work with backend engineer to help translate analyses into software products.

This takehome is your opportunity to demonstrate that you have the right technical skills to accomplish that. 

We have received applications from candidates with a variety of backgrounds. While previous experience will factor into our overall assessment of the candidate, performance on this takehome will play a strong role in our final decision. The Recycling Partnership is a team of growth-oriented people across all verticals of the organization and we hire accordingly. 

## Some things we're looking for

* Code is read more than it is written, so write code that is easy to read! Things like documentation in markdown cells, docstrings, type hinting, and descriptive names for variables and functions are appreciated.
* It's a messy world out there. Where appropriate, demonstrate good practices for data validation and error handling.
* Show off! If there is a cool tool or technique that you want to use, go for it. Make notes of any special libraries to be installed, along with versions.

**We expect that this should take around 2 to 4 hours to complete. Be creative, but don't work too hard.**

# Part 1. Database interaction

## Introducing the data

I put together a little SQLite database to use as a demo. It is distributed with this notebook, called `veterinary_clinic.sqlite`. It contains a bit of data for an imaginary veterinary clinic. In the database, there are five tables:
- `pet_type`: an enumeration of pet types like: `'dog'`, `'cat'`, etc
- `veterinarian`: the veterinarians in the clinic's practice
- `person`: the people who provide homes and care for the pets
- `pet`: the pets who are patients at the clinic
- `pet_person`: a join table between Persons and Pets, a many-to-many relationship

Here is the schema for each of the tables:

### `pet_type`

```SQL
create table pet_type
(
    id    INTEGER not null
        constraint pet_type_pk
            primary key autoincrement,
    value TEXT    not null
);
```

### `veterinarian`

```SQL
create table veterinarian
(
    id   INTEGER not null
        constraint veterinarian_pk
            primary key autoincrement,
    name TEXT    not null
);

```

### `person`

```SQL
create table person
(
    id    INTEGER not null
        constraint person_pk
            primary key autoincrement,
    name  TEXT    not null,
    email TEXT,
    phone TEXT
);
```

### `pet`

```SQL
create table pet
(
    id                      INTEGER not null
        constraint pet_pk
            primary key autoincrement,
    name                    TEXT    not null,
    date_of_birth           TEXT    not null,  -- ISO formatted dates, like: 2020-12-31
    alive                   INTEGER default 1 not null, -- boolean: 1 = True, 0 = False
    pet_type_id             INTEGER not null
        references pet_type,
    primary_veterinarian_id INTEGER not null
        references veterinarian
);

create index pet_pet_type_id_index
    on pet (pet_type_id);

create index pet_primary_veterinarian_id_index
    on pet (primary_veterinarian_id);
```

### `pet_person`

```SQL
create table pet_person
(
    pet_id    INTEGER not null
        references pet,
    person_id INTEGER not null
        references person,
    primary key (pet_id, person_id)
);
```

## What you should do

The aim of this part is to show how you would approach the data and doing the analysis. Feel free to use as much code/space/cells as you like, just try to make sure that the notebook is runnable, repeatable. Here are some questions we'd like to see your analysis answer:

1. For all of the pet types at this clinic, what is the average age of each pet type (only including pets which are currently alive)? Please give the answer in days.
1. Which veterinarian at the clinic has the most patients?
1. Which veterinarian at the clinic sees the most "exotic" pets? For the purposes of this problem, consider "exotic" to mean any pet type which is not `'cat'` or `'dog'`. 
1. Spot the clerical error. Someone forgot to assign a person to one of the pets. Which pet has no person?
1. Make a "call-sheet" CSV. Imagine the clinic needs to get in touch with their client Persons for an important announcement. The staff will use this sheet to make the calls. Include the following columns:
    1. client_id: `person.id`
    1. client_name: `person.name`
    1. phone: `person.phone`
    1. email: `person.email`
    1. pets: A comma-separated list of each of their living pet's names. example: `'Josie, Eddie, Mercury, Chandra'`
    1. vets: A comma-separated list of each of their living pets's primary vets. Often only one, but may be more.

## Tips

Python has a standard library module for working with SQLite: `sqlite3`. The documentation for it is here: https://docs.python.org/3/library/sqlite3.html. A quickstart piece of code to use that could look like this:

```python
import sqlite3

cursor = sqlite3.connect("veterinary_clinic.sqlite").cursor()

pet_type_rows = cursor.execute("SELECT * FROM pet_type;").fetchall()
print(pet_type_rows)
```

But we encourage you to use whatever other tools you like. We frequently make use of SQLAlchemy. 

In [ ]:
# Your code here

# Part 2 - Code Review

The following snippet of code comes from an API built using [FastAPI](https://fastapi.tiangolo.com/) and implements an endpoint to create a Product in some system. Products are required to have unique names, but the names are not used as the primary key in the database. This endpoint tries to provide a helpful error message in case someone tries to create a Product with a name that is already in use.

Please review and critique this code. Point out any bugs or ways to improve the code, its clarity, and/or its behavior.

In [ ]:
from database_models import Product as DbProduct  # assume this is custom library we provide

# an asynchronous context manager for a database session. See:
#   https://docs.sqlalchemy.org/en/20/orm/extensions/asyncio.html#sqlalchemy.ext.asyncio.AsyncSession
#   https://docs.sqlalchemy.org/en/20/orm/session_basics.html
from utils.db_sessions import db_session_context  # assume this is custom library we provide


from fastapi import APIRouter, HTTPException
import pydantic
from sqlalchemy import select


router = APIRouter(
    prefix="/products",
    tags=["Products"],
    responses={404: {"description": "Not found"}},
)


class ProductCreate(pydantic.BaseModel):
    name: str
    sku: str
    notes: str | None


class Product(pydantic.BaseModel):
    id: int
    name: str
    sku: str
    notes: str | None


async def fetch_product(db_session, name) -> DbProduct | None:
    product_query = select(DbProduct).where(DbProduct.name == name)
    product = (await db_session.execute(product_query)).scalars().first()
    return product


@router.post("/create", response_model=Product)
async def create_product(product_to_create: ProductCreate) -> Product:
    """
    Create a new product.
    """
    async with db_session_context() as db_session:
        product = await fetch_product(db_session, product_to_create.name)
        if product is None:
            product = DbProduct(name=product_to_create.name, sku=product_to_create.sku, notes=product_to_create.notes or None)
            db_session.add(product)
            await db_session.commit()
            await db_session.refresh(product)
        else:
            raise HTTPException(
                400,
                detail={
                    "error_type": "Product Name already in use",
                    "product_name": product.name,
                },
            )
    return Product(id=product.id, name=product.name, sku=product.sku, notes=product.notes)


*Your answer here*

# Part 3 - Monitoring and processing unstructured web data

We want to monitor for changes to municipal websites containing recycling guidelines and instructions for their community. In a few sentences, describe how you might develop a process to detect changes to guidelines while trying to avoid false positive alerts. Some examples of such municipal websites are given below:

- San Francisco: https://www.recology.com/recology-san-francisco/your-three-carts/
- Garland, TX: https://www.garlandtx.gov/448/Acceptable-Materials
- Summit County, OH: https://www.summitreworks.com/239/What-Can-Be-Recycled-in-Curbside-Carts
- Mercer County, OH: https://www.mercercountyohio.org/county-services/solid-waste-management/recycling/
- Kittery, ME: https://www.kitteryme.gov/resource-recovery-facility-transfer-station

*Your answer here...*